## Calculate ATO for TAZ centroid(s)

1. Run 1_setup_network.ipynb to create NetworkDataset_MM
2. Create OD Cost Matrix Layer
3. Add origin(s)
4. Add destinations
5. Solve
6. Join attributes to solved Lines layer
7. Weight HH and JOB by time decay
8. Sum

In [ ]:
# Set to True to limit to ~30 TAZs
testing = False

#input = "auto_baseline"
input = "bus_baseline"
#input = r"scenario\auto_widen_12300"

In [ ]:
input = "mod"
mode = "Transit"
network = r"scenario\bus_sandy_circulator.gdb"
output_file = input

In [ ]:
import arcpy
import os

arcpy.CheckOutExtension("network")

from arcgis.features import SpatialDataFrame
import pandas as pd
# from arcgis.features import GeoAccessor, GeoSeriesAccessor

def survey_weight(t):
    if t <= 3:
        return 1
    elif (t > 3) & (t <= 20):
        return -0.0382 * t + 1.1293
    elif t > 20:
        return 1/(1 + math.exp(0.1092 * t - 1.5604))
    else:
        return 0

base_path = os.path.abspath(".")
network = os.path.join(base_path, network, r"NetworkDataset\NetworkDataset_ND")

arcpy.env.workspace = os.path.join(base_path, "ato.gdb")

In [ ]:
nd_path = network
nd_layer_name = "wfrc_mm"
arcpy.nax.MakeNetworkDatasetLayer(nd_path, nd_layer_name)

In [ ]:
if testing:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_sample")
else:
    centroids = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_snapped")

In [ ]:
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)

In [ ]:
odcm.travelMode = mode
odcm.defaultImpedanceCutoff = 60
odcm.lineShapeType = arcpy.nax.LineShapeType.NoLine

In [ ]:
# Load inputs using field mappings, including the pre-calculated location fields
field_mappings_origins = odcm.fieldMappings(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, True)
# Map the "CO_TAZID" field in the input data to the "Name" property of the Origins class
field_mappings_origins["Name"].mappedFieldName = "CO_TAZID"
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, centroids, field_mappings_origins)
# Load destinations
field_mappings_destinations = odcm.fieldMappings(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, True)
field_mappings_destinations["Name"].mappedFieldName = "CO_TAZID"
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, centroids, field_mappings_destinations)

In [ ]:
%%time
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
if result.solveSucceeded:
    result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, "memory\output_lines")
else:
    print("Solve failed")
    print(result.solverMessages(arcpy.nax.MessageSeverity.All))

In [ ]:
od = pd.DataFrame.spatial.from_featureclass(r"memory\output_lines")

In [ ]:
od.head()

In [ ]:
if od['Total_Time'].mean() < 1:
    print("Network validation: FAIL")
    print("Travel Times: ", od['Total_Time'].head())
    raise ValueError('Network Travel Times are Zero - Invalid Network')

In [ ]:
taz = pd.DataFrame.spatial.from_featureclass(os.path.join(r"shp\taz_wfrc.gdb", "ATO"))

In [ ]:
taz = taz[['CO_TAZID', 'HH_19', 'JOB_19', 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19']]

In [ ]:
od['DestinationName'] = od['DestinationName'].astype(int)

In [ ]:
df = pd.merge(od, taz, left_on="DestinationName", right_on="CO_TAZID", )

In [ ]:
df.rename(columns={"OriginName": "Origin_TAZID",
                   "DestinationName": "Destination_TAZID"},
          inplace=True)

In [ ]:
# Weight outputs
df['survey_weight'] = df['Total_Time'].apply(lambda x: survey_weight(x)).round(3)


df['weighted_jobs'] = df['survey_weight'] * df['JOB_19']
df['weighted_hh'] = df['survey_weight'] * df['HH_19']
df['weighted_jobs'] = round(df['weighted_jobs'])
df['weighted_hh'] = round(df['weighted_hh'])

df['ato'] = df['weighted_jobs'] + df['weighted_hh']

In [ ]:
# write to disk
df[['Origin_TAZID', 'Destination_TAZID', 'Total_Time',
         'weighted_jobs', 'weighted_hh', 'ato']].to_csv(output_file + '.csv')

In [ ]:
taz_summary = df.groupby('Origin_TAZID').agg(
    jobs=pd.NamedAgg(column='weighted_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='weighted_hh', aggfunc=sum)
)
taz_summary['ato'] = taz_summary['jobs'] + taz_summary['hh']
taz_summary.to_csv(output_file + '_summary.csv')